## Set up

In [ ]:
!pip install git+https://github.com/jameschamberlain/bigbird.git -q

     |████████████████████████████████| 1.2MB 13.0MB/s 
     |████████████████████████████████| 3.4MB 47.8MB/s 
     |████████████████████████████████| 1.5MB 52.7MB/s 
     |████████████████████████████████| 3.9MB 53.9MB/s 
     |████████████████████████████████| 983kB 52.0MB/s 
     |████████████████████████████████| 358kB 53.6MB/s 
     |████████████████████████████████| 5.6MB 45.4MB/s 
     |████████████████████████████████| 368kB 54.3MB/s 
     |████████████████████████████████| 194kB 54.3MB/s 
     |████████████████████████████████| 378kB 48.8MB/s 
     |████████████████████████████████| 706kB 48.1MB/s 
     |████████████████████████████████| 655kB 51.4MB/s 
     |████████████████████████████████| 368kB 47.4MB/s 
     |████████████████████████████████| 256kB 52.3MB/s 
ERROR: kfac 0.2.3 has requirement tensorflow-probability==0.8, but you'll have tensorflow-probability 0.7.0 which is incompatible.


In [ ]:
from bigbird.core import flags
from bigbird.core import modeling
from bigbird.core import utils
from bigbird.question_generation import run_question_generation
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_text as tft
from tqdm import tqdm
import sys

FLAGS = flags.FLAGS
if not hasattr(FLAGS, "f"): flags.DEFINE_string("f", "", "")
FLAGS(sys.argv)

tf.enable_v2_behavior()

## Set options

In [ ]:
FLAGS.data_dir = "tfds://sqa_qg_dataset"
FLAGS.attention_type = "original_full"
FLAGS.couple_encoder_decoder = True
FLAGS.max_encoder_length = 256  # on free colab only lower memory GPU like T4 is available
FLAGS.max_decoder_length = 128
FLAGS.block_size = 64
FLAGS.learning_rate = 1e-5
FLAGS.num_train_steps = 10000
FLAGS.attention_probs_dropout_prob = 0.0
FLAGS.hidden_dropout_prob = 0.0
FLAGS.use_gradient_checkpointing = True
FLAGS.vocab_model_file = "gpt2"

In [ ]:
transformer_config = flags.as_dictionary()

## Define question generation model

In [ ]:
from tensorflow.python.ops.variable_scope import EagerVariableStore
container = EagerVariableStore()

In [ ]:
with container.as_default():
  model = modeling.TransformerModel(transformer_config)

In [ ]:
@tf.function(experimental_compile=True)
def fwd_bwd(features, labels):
  with tf.GradientTape() as g:
    (llh, logits, pred_ids), _ = model(features, target_ids=labels,
                                       training=True)
    loss = run_question_generation.padded_cross_entropy_loss(
        logits, labels,
        transformer_config["label_smoothing"],
        transformer_config["vocab_size"])
  grads = g.gradient(loss, model.trainable_weights)
  return loss, llh, logits, pred_ids, grads

## Dataset pipeline

In [ ]:
train_input_fn = run_question_generation.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        max_decoder_length=FLAGS.max_decoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=True,
        tmp_dir='gs://q-gen/tensorflow_datasets')
dataset = train_input_fn({'batch_size': 8})

/usr/local/lib/python3.7/dist-packages/bigbird/question_generation/run_question_generation.py:184: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  deterministic=is_training)


## Check outputs

In [ ]:
loss, llh, logits, pred_ids, grads = fwd_bwd(ex[0], ex[1])
print('Loss: ', loss)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Loss:  tf.Tensor(45.25094, shape=(), dtype=float32)


## (Optionally) Load pretrained model

In [ ]:
# For training from scratch use
# ckpt_path = 'gs://bigbird-transformer/pretrain/bigbr_base/model.ckpt-0'
# For quick check continue from trained checkpoint
ckpt_path = 'gs://bigbird-transformer/summarization/pubmed/roberta/model.ckpt-300000'
ckpt_reader = tf.compat.v1.train.NewCheckpointReader(ckpt_path)
loaded_weights = []
for v in tqdm(model.trainable_weights, position=0):
  try:
    val = ckpt_reader.get_tensor(v.name[:-2])
  except:
    val = v.numpy()
  loaded_weights.append(val)

model.set_weights(loaded_weights)

100%|██████████| 316/316 [00:00<00:00, 417.43it/s]


## Train

In [ ]:
opt = tf.keras.optimizers.Adam(FLAGS.learning_rate)
train_loss = tf.keras.metrics.Mean(name='train_loss')

for i, ex in enumerate(tqdm(dataset.take(FLAGS.num_train_steps), position=0)):
  loss, llh, logits, pred_ids, grads = fwd_bwd(ex[0], ex[1])
  opt.apply_gradients(zip(grads, model.trainable_weights))
  train_loss(loss)
  if i% 10 == 0:
    print('Loss = {} '.format(train_loss.result().numpy()))

  0%|          | 1/10000 [00:03<9:08:06,  3.29s/it]

Loss = 52.736576080322266 


  0%|          | 11/10000 [00:05<50:18,  3.31it/s]

Loss = 22.46729278564453 


  0%|          | 21/10000 [00:07<36:07,  4.60it/s]

...

100%|█████████▉| 9981/10000 [36:32<00:04,  4.55it/s]

Loss = 4.670989036560059 


100%|█████████▉| 9991/10000 [36:35<00:01,  4.54it/s]

Loss = 4.670675754547119 


100%|██████████| 10000/10000 [36:37<00:00,  4.55it/s]


## Eval

In [ ]:
@tf.function(experimental_compile=True)
def fwd_only(features, labels):
  (llh, logits, pred_ids), _ = model(features, target_ids=labels,
                                       training=False)
  return llh, logits, pred_ids

In [ ]:
eval_input_fn = run_summarization.input_fn_builder(
        data_dir=FLAGS.data_dir,
        vocab_model_file=FLAGS.vocab_model_file,
        max_encoder_length=FLAGS.max_encoder_length,
        max_decoder_length=FLAGS.max_decoder_length,
        substitute_newline=FLAGS.substitute_newline,
        is_training=False)
eval_dataset = eval_input_fn({'batch_size': 8})

DatasetNotFoundError: ignored

In [ ]:
eval_llh = tf.keras.metrics.Mean(name='eval_llh')

for ex in tqdm(eval_dataset, position=0):
  llh, logits, pred_ids = fwd_only(ex[0], ex[1])
  eval_llh(llh)
print('Log Likelihood = {}'.format(eval_llh.result().numpy()))

### Print predictions

In [ ]:
tokenizer = tft.SentencepieceTokenizer(
        model=tf.io.gfile.GFile(FLAGS.vocab_model_file, "rb").read())

In [ ]:
_, _, pred_ids = fwd_only(ex[0], ex[1])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [ ]:
print('Context:\n {}\n\n Predicted question:\n {}\n\n Ground truth question:\n {}\n\n'.format(
    tokenizer.detokenize(ex[0]),
    tokenizer.detokenize(pred_ids),
    tokenizer.detokenize(ex[1])))

Context:
 [b'it regards its target countries as the middle east but adopts the convention of calling them the near east to be in conformance with the practices of the state department .'
 b'the football team has won 13 sec championships and 25 bowls , including four sugar bowls , three cotton bowls , an orange bowl and a fiesta bowl .']

 Predicted question:
 [b'what is the name of the ?' b'what is the name of the ?']

 Ground truth question:
 [b'what is wineps target countries as ?'
 b'how many college football bowl championships have the tennessee volunteers won ?']


